In [1]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover

In [2]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
data = yf.download(ticker, period="1y", interval="1h")


[*********************100%%**********************]  1 of 1 completed


In [4]:
# Importing through a CSV that has more data which was downloaded externally.
df = pd.read_csv('../Data/AUDCAD_H1(22007-09-26 - 2024-02-02).csv', delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0)
df.index = pd.to_datetime(df.index)


In [ ]:
# csv slice by date
start_date = '2010-01-01'
end_date = '2024-01-01'
df = df.loc[start_date:end_date]

# Write out strategy here to figure out the logic

In [ ]:
class Strat(Strategy):
    def init(self):
        pass

    def next(self):
        pass

bt = Backtest(df_slice, Strat, cash=10_000)
bt.run()
bt.plot()

In [16]:
help(max)

Help on built-in function max in module builtins:

max(...)
    max(iterable, *[, default=obj, key=func]) -> value
    max(arg1, arg2, *args, *[, key=func]) -> value
    
    With a single iterable argument, return its biggest item. The
    default keyword-only argument specifies an object to return if
    the provided iterable is empty.
    With two or more arguments, return the largest argument.



In [21]:
class Strat(Strategy):
    def init(self):
        self.ema = self.I(ta.ema, pd.Series(self.data.Close), length=50)
        self.atr = self.I(ta.atr, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close), length=14)
        self.pullback = False
        self.pullback_count = 0
        self.consolidation_high = 0
        
    def next(self):
        current_price = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]
        
        if current_high > self.data.High[-2]:
            self.consolidation_high = max(self.consolidation_high, current_high)       
       
        # Bullish Pullback
        if current_price > self.ema[-1]:
            if current_price < current_open: self.pullback_count += 1
            if self.pullback_count >= 2:
                self.pullback = True
            if self.pullback:
                if current_price > self.data.High[-2] and current_price < self.consolidation_high:
                    self.buy(sl=(self.data.Low[-1] - self.atr), tp=(self.data.High[-1] + (self.atr * 1.2)))
                    self.pullback_count = 0
                    self.pullback = False
                    self.consolidation_high = 0

In [22]:
bt = Backtest(data, Strat, cash=100000)
stats = bt.run()
print(stats)
bt.plot()

Start                     2023-02-06 19:00...
End                       2024-02-06 19:00...
Duration                    365 days 00:00:00
Exposure Time [%]                    45.77568
Equity Final [$]                 99314.700572
Equity Peak [$]                 103341.499351
Return [%]                          -0.685299
Buy & Hold Return [%]                11.62949
Return (Ann.) [%]                   -0.659229
Volatility (Ann.) [%]                5.098277
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.966947
Avg. Drawdown [%]                   -0.589792
Max. Drawdown Duration      169 days 19:00:00
Avg. Drawdown Duration       14 days 07:00:00
# Trades                                  274
Win Rate [%]                        57.664234
Best Trade [%]                       0.590568
Worst Trade [%]                     -1.066632
Avg. Trade [%]                    

d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p3320', ...)

In [26]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,760,78,84,131.557007,131.218608,-257.183063,-0.002572,2023-02-10 01:00:00+00:00,2023-02-10 07:00:00+00:00,0 days 06:00:00
1,759,93,100,131.332993,131.915947,442.462209,0.004439,2023-02-10 16:00:00+00:00,2023-02-13 01:00:00+00:00,2 days 09:00:00
2,760,100,103,131.694000,132.150605,347.019308,0.003467,2023-02-13 01:00:00+00:00,2023-02-13 04:00:00+00:00,0 days 03:00:00
3,759,108,113,132.436996,132.859620,320.771390,0.003191,2023-02-13 09:00:00+00:00,2023-02-13 14:00:00+00:00,0 days 05:00:00
4,763,131,136,132.078995,131.564822,-392.314082,-0.003893,2023-02-14 08:00:00+00:00,2023-02-14 13:00:00+00:00,0 days 05:00:00
...,...,...,...,...,...,...,...,...,...,...
269,676,5978,5989,148.141998,147.727569,-280.154522,-0.002798,2024-01-26 19:00:00+00:00,2024-01-29 08:00:00+00:00,2 days 13:00:00
270,675,5993,5997,147.869995,147.599957,-182.275558,-0.001826,2024-01-29 12:00:00+00:00,2024-01-29 16:00:00+00:00,0 days 04:00:00
271,675,6032,6043,147.675995,147.335838,-229.606107,-0.002303,2024-01-31 03:00:00+00:00,2024-01-31 14:00:00+00:00,0 days 11:00:00
272,678,6086,6090,146.731003,147.016173,193.345630,0.001943,2024-02-02 09:00:00+00:00,2024-02-02 13:00:00+00:00,0 days 04:00:00


In [ ]:

# plot own equity curve based on backtesting data for data more than 10K
equity_curve = stats._equity_curve['Equity']

plt.figure(figsize=(10, 6))
plt.plot(equity_curve, label='Equity Curve', lw=1)  # lw is line width
plt.title('Equity Curve')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(False)
plt.show()


In [ ]:

equity_curve = stats._equity_curve['Equity'].reset_index()

fig = px.line(equity_curve, x='index', y='Equity', labels={'index': 'Time'}, title='Strategy Performance')
fig.update_layout(height=600, xaxis_title='Time', yaxis_title='Equity', legend_title='Legend')
fig.show()
